In [1]:
FILENAME = "000000.pcd" # The file to segment
SUBSAMPLE = 5 # The amoutn of subsampling for faster plotting

In [2]:
%%capture
%matplotlib inline
import os
import numpy as np 
import open3d as o3d
import matplotlib.cm as cm
from ipywidgets import interact, widgets
from IPython.display import display
import matplotlib.pyplot as plt
import py_pcl_seg_lib

plt.rcParams['figure.dpi'] = 600

in_file = os.path.join(os.path.realpath(globals()['_dh'][0]), FILENAME)

pcd = o3d.io.read_point_cloud(in_file)
out_arr = np.asarray(pcd.points)  
cloud_x, cloud_y, cloud_z= list(zip(*out_arr[::SUBSAMPLE]))

def interactive_segment(max_iters, dist_thresh_cm, model, sac_type):
    result = py_pcl_seg_lib.ground_segment_cloud_from_file(
                            in_file,
                            in_file + ".ground.pcd",
                            in_file + ".outliers.pcd",
                            model,
                            sac_type,
                            int(max_iters),
                            dist_thresh_cm * 0.01,
                            2,
                            5.0 * (3.14159/180.0)
                        )
    ground_pcd = o3d.io.read_point_cloud(in_file + ".ground.pcd")
    ground_out_arr = np.asarray(ground_pcd.points)  
    ground_cloud_x, ground_cloud_y, ground_cloud_z= list(zip(*ground_out_arr[::SUBSAMPLE]))
    
    ax_seg_top.clear()
    ax_seg_top.scatter(ground_cloud_x, ground_cloud_y, s=0.1, c=cm.coolwarm([5*z for z in ground_cloud_z]/max(cloud_z)))
    ax_seg_top.set_aspect('equal')
    plt.setp(ax_seg_top, xlim=ax_orig.get_xlim(), ylim=ax_orig.get_ylim())
    
    ax_seg_side.clear()
    ax_seg_side.scatter(ground_cloud_x, ground_cloud_z, s=0.05, c=cm.coolwarm([5*z for z in ground_cloud_z]/max(cloud_z)))
    ax_seg_side.set_aspect('equal')
    
    ax_seg_top.set_title('Seg. Top')
    ax_seg_side.set_title('Seg. Side')
    plt.setp(ax_seg_top, xlim=ax_orig.get_xlim(), ylim=ax_orig.get_ylim())
    plt.setp(ax_seg_side, xlim=ax_orig.get_xlim(), ylim=ax_orig.get_ylim())

    fig.canvas.draw()
    display(fig)

fig, axs = plt.subplots(2, 2)
ax_orig = axs[0,0]
ax_orig_side = axs[1,0]
ax_seg_top = axs[0,1] 
ax_seg_side = axs[1,1] 
ax_orig.set_title('Top')
ax_orig_side.set_title('Side')
ax_orig.scatter(cloud_x, cloud_y, s=0.05, c=cm.coolwarm([5*z for z in cloud_z]/max(cloud_z)))
ax_orig_side.scatter(cloud_x, cloud_z, s=0.05, c=cm.coolwarm([5*z for z in cloud_z]/max(cloud_z)))
ax_orig.set_aspect('equal')
plt.setp(ax_orig, xlim=(-45,45), ylim=(-30,30))
plt.setp(ax_orig_side, xlim=ax_orig.get_xlim(), ylim=ax_orig.get_ylim())

In [3]:
# 'SAC_RMSAC' is extremely slow, we leave it out.
interact(interactive_segment, 
         max_iters=widgets.IntSlider(min=20,max=1000,step=10,value=10), 
         dist_thresh_cm=widgets.IntSlider(min=0,max=200,step=1,value=5),
         model=widgets.Dropdown(options=['SACMODEL_PERPENDICULAR_PLANE', 'SACMODEL_PLANE']),
        sac_type=widgets.Dropdown(options=['SAC_RANSAC', 'SAC_LMEDS', 'SAC_MSAC', 'SAC_RRANSAC', 'SAC_MLESAC', 'SACi_PROSAC']));

interactive(children=(IntSlider(value=20, description='max_iters', max=1000, min=20, step=10), IntSlider(value…